In [11]:
# !gdown 1qq5gDuEAqWnoviRBDYfbprLFD-MPQyNo
# !gdown 1mq9CWT9MXsH1Y-ihxVL4uEzufQjpBoer
!gdown 1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql

Downloading...
From: https://drive.google.com/uc?id=1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql
To: /content/processed_data.zip
100% 5.62M/5.62M [00:00<00:00, 33.1MB/s]


In [12]:
!rm -rf processed_data

!unzip -q processed_data.zip

In [13]:
!pip install -q transformers[torch] accelerate -U

!pip install -q datasets

!pip install -q transformers

!pip install -q iterative-stratification

!pip install -q auto-gptq optimum bitsandbytes

In [14]:
import numpy as np

import pandas as pd

import os

from os import path

import glob

from tqdm.notebook import tqdm

from tqdm import tqdm

import string

import copy

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,IntervalStrategy,BitsAndBytesConfig

from datasets import Dataset,load_dataset, DatasetDict

# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



from sklearn.model_selection import train_test_split,KFold

from sklearn.metrics import classification_report,accuracy_score, f1_score

from sklearn.metrics import precision_recall_fscore_support

from sklearn.utils import shuffle

from sklearn.preprocessing import LabelEncoder

from scipy.special import expit

from peft import prepare_model_for_kbit_training

from peft import LoraConfig, get_peft_model,PeftModel


In [15]:
import torch

import torch.nn as nn

import torch.nn.functional as F

In [16]:


models_list_3=[
        "FacebookAI/xlm-roberta-large",
      # "google-bert/bert-base-uncased",

#      "google-bert/bert-base-multilingual-uncased",

          ]



models_name_3=[
"FacebookAI/xlm-roberta-large",
# "google-bert/bert-base-uncased",

# "bert",

]


In [17]:
kind="train"

task="track_a"

# langs=["afr","amh","deu","eng","oro","ptbr","rus","som","sum","tir"]
# labels=['Anger','Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
# langs=["amh","arq","ary","chn","deu","esp","hau","hin","ibo","kin","mar","orm","pcm","ptbr","ptmz","ron","rus","som","sun","swa","swe","tat","tir","ukr","vmw","yor","ind","jav","xho","zul"]
langs=["eng"]
# langs=["afr"]


# processed_path=f"processed_data/{kinds[0]}/{tasks[0]}/{langs[3]}.csv"



# train_data=pd.read_csv(processed_path)


In [18]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    # if kind=="dev":
    #    processed_path=processed_path.replace(lang,lang+task[-2:])
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)




In [19]:
all_csv

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_train_track_a_00001,0,1,0,0,1,eng,colorado middle of nowhere
1,eng_train_track_a_00002,0,1,0,0,0,eng,this involved swimming a pretty large lake tha...
2,eng_train_track_a_00003,0,1,0,1,0,eng,it was one of my most shameful experiences
3,eng_train_track_a_00004,0,0,0,0,0,eng,after all i had vegetables coming out my ears ...
4,eng_train_track_a_00005,0,1,0,1,1,eng,then the screaming started
...,...,...,...,...,...,...,...,...
2763,eng_train_track_a_02764,0,0,1,0,1,eng,she cants her hip against my waist into my sid...
2764,eng_train_track_a_02765,0,0,0,0,0,eng,i then did the dishes whitened my teeth watche...
2765,eng_train_track_a_02766,0,0,0,0,1,eng,it just kind of gradually vanished over a coup...
2766,eng_train_track_a_02767,0,1,0,0,0,eng,i didnt look out of my hands


In [20]:
all_csv[all_csv["lang"]=="eng"]

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_train_track_a_00001,0,1,0,0,1,eng,colorado middle of nowhere
1,eng_train_track_a_00002,0,1,0,0,0,eng,this involved swimming a pretty large lake tha...
2,eng_train_track_a_00003,0,1,0,1,0,eng,it was one of my most shameful experiences
3,eng_train_track_a_00004,0,0,0,0,0,eng,after all i had vegetables coming out my ears ...
4,eng_train_track_a_00005,0,1,0,1,1,eng,then the screaming started
...,...,...,...,...,...,...,...,...
2763,eng_train_track_a_02764,0,0,1,0,1,eng,she cants her hip against my waist into my sid...
2764,eng_train_track_a_02765,0,0,0,0,0,eng,i then did the dishes whitened my teeth watche...
2765,eng_train_track_a_02766,0,0,0,0,1,eng,it just kind of gradually vanished over a coup...
2766,eng_train_track_a_02767,0,1,0,0,0,eng,i didnt look out of my hands


In [21]:
# float_columns = all_csv.select_dtypes(include=['float64']).columns
# # Check each column and convert only if there are no NaN values
# for col in float_columns:
#     if all_csv[col].isna().sum() == 0:  # No NaN values in the column
#         print(f"converting {kind} {task} {lang} column {col} to int")
#         all_csv[col] = all_csv[col].astype('int')
#     else:
#         print(f"converting {kind} {task} {lang} column {col} to int")

In [22]:
train_data=all_csv


In [23]:
# train_data = train_data.drop(columns=['disgust'],axis=1)


In [24]:

train_data=train_data.fillna(0)

train_data

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_train_track_a_00001,0,1,0,0,1,eng,colorado middle of nowhere
1,eng_train_track_a_00002,0,1,0,0,0,eng,this involved swimming a pretty large lake tha...
2,eng_train_track_a_00003,0,1,0,1,0,eng,it was one of my most shameful experiences
3,eng_train_track_a_00004,0,0,0,0,0,eng,after all i had vegetables coming out my ears ...
4,eng_train_track_a_00005,0,1,0,1,1,eng,then the screaming started
...,...,...,...,...,...,...,...,...
2763,eng_train_track_a_02764,0,0,1,0,1,eng,she cants her hip against my waist into my sid...
2764,eng_train_track_a_02765,0,0,0,0,0,eng,i then did the dishes whitened my teeth watche...
2765,eng_train_track_a_02766,0,0,0,0,1,eng,it just kind of gradually vanished over a coup...
2766,eng_train_track_a_02767,0,1,0,0,0,eng,i didnt look out of my hands


In [25]:
float_columns = train_data.select_dtypes(include=['float64']).columns

# Convert those columns to int
train_data[float_columns] = train_data[float_columns].astype('int')
train_data

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_train_track_a_00001,0,1,0,0,1,eng,colorado middle of nowhere
1,eng_train_track_a_00002,0,1,0,0,0,eng,this involved swimming a pretty large lake tha...
2,eng_train_track_a_00003,0,1,0,1,0,eng,it was one of my most shameful experiences
3,eng_train_track_a_00004,0,0,0,0,0,eng,after all i had vegetables coming out my ears ...
4,eng_train_track_a_00005,0,1,0,1,1,eng,then the screaming started
...,...,...,...,...,...,...,...,...
2763,eng_train_track_a_02764,0,0,1,0,1,eng,she cants her hip against my waist into my sid...
2764,eng_train_track_a_02765,0,0,0,0,0,eng,i then did the dishes whitened my teeth watche...
2765,eng_train_track_a_02766,0,0,0,0,1,eng,it just kind of gradually vanished over a coup...
2766,eng_train_track_a_02767,0,1,0,0,0,eng,i didnt look out of my hands


In [26]:
train_data.dtypes

,0
id,object
anger,int64
fear,int64
joy,int64
sadness,int64
surprise,int64
lang,object
clean_message,object


In [27]:
train_data = train_data[

train_data['clean_message'].isnull()==False]

train_data.reset_index()

,index,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,0,eng_train_track_a_00001,0,1,0,0,1,eng,colorado middle of nowhere
1,1,eng_train_track_a_00002,0,1,0,0,0,eng,this involved swimming a pretty large lake tha...
2,2,eng_train_track_a_00003,0,1,0,1,0,eng,it was one of my most shameful experiences
3,3,eng_train_track_a_00004,0,0,0,0,0,eng,after all i had vegetables coming out my ears ...
4,4,eng_train_track_a_00005,0,1,0,1,1,eng,then the screaming started
...,...,...,...,...,...,...,...,...,...
2763,2763,eng_train_track_a_02764,0,0,1,0,1,eng,she cants her hip against my waist into my sid...
2764,2764,eng_train_track_a_02765,0,0,0,0,0,eng,i then did the dishes whitened my teeth watche...
2765,2765,eng_train_track_a_02766,0,0,0,0,1,eng,it just kind of gradually vanished over a coup...
2766,2766,eng_train_track_a_02767,0,1,0,0,0,eng,i didnt look out of my hands


In [28]:
train_data

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_train_track_a_00001,0,1,0,0,1,eng,colorado middle of nowhere
1,eng_train_track_a_00002,0,1,0,0,0,eng,this involved swimming a pretty large lake tha...
2,eng_train_track_a_00003,0,1,0,1,0,eng,it was one of my most shameful experiences
3,eng_train_track_a_00004,0,0,0,0,0,eng,after all i had vegetables coming out my ears ...
4,eng_train_track_a_00005,0,1,0,1,1,eng,then the screaming started
...,...,...,...,...,...,...,...,...
2763,eng_train_track_a_02764,0,0,1,0,1,eng,she cants her hip against my waist into my sid...
2764,eng_train_track_a_02765,0,0,0,0,0,eng,i then did the dishes whitened my teeth watche...
2765,eng_train_track_a_02766,0,0,0,0,1,eng,it just kind of gradually vanished over a coup...
2766,eng_train_track_a_02767,0,1,0,0,0,eng,i didnt look out of my hands


In [42]:
label_columns =['anger', 'fear', 'joy', 'sadness', 'surprise']
# label_columns =['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']


In [43]:
train_data['lang'].value_counts()

,count
lang,
eng,2768


In [44]:
sum(train_data['lang'].isna())

0

In [45]:
nan_rows = train_data[train_data.isna().any(axis=1)]
nan_rows
# Group by 'lang' and list the rows with NaNs for each language
# nan_by_language = nan_rows.groupby('lang').apply(lambda x: x)

# Display the result
# print(nan_by_language)

,id,anger,fear,joy,sadness,surprise,lang,clean_message


In [46]:
nan_rows[nan_rows["lang"]=="eng"]

,id,anger,fear,joy,sadness,surprise,lang,clean_message


In [47]:
train_data[train_data["lang"]=="deu"].dtypes

,0
id,object
anger,int64
fear,int64
joy,int64
sadness,int64
surprise,int64
lang,object
clean_message,object


In [48]:
nan_rows["lang"].value_counts()

,count
lang,


In [49]:
nan_rows.dtypes

,0
id,object
anger,int64
fear,int64
joy,int64
sadness,int64
surprise,int64
lang,object
clean_message,object


In [50]:
# Step 2: Count how many columns are not 0 or 1 for each language
def count_nonzero_nonone_rows(group):
    # Exclude the 'lang' column from counting if needed
    # Use boolean conditions to count values that are not 0 or 1
    return (group.drop(columns='anger') != 0) & (group.drop(columns='anger') != 1).sum(axis=1).sum()

# Group by 'lang' and apply the function to count non-zero, non-one values
non_zero_non_one_counts = train_data.groupby('lang').apply(count_nonzero_nonone_rows)

# Display the results
# result = pd.DataFrame({
#     'row_count': lang_row_counts,
#     'non_zero_non_one_count': non_zero_non_one_counts
# })

print(non_zero_non_one_counts)

              id   fear    joy  sadness  surprise   lang  clean_message
lang                                                                   
eng  1374  False  False  False    False     False  False          False
     120   False  False  False    False     False  False          False
     2017  False  False  False    False     False  False          False
     2588  False  False  False    False     False  False          False
     1461  False  False  False    False     False  False          False
...          ...    ...    ...      ...       ...    ...            ...
     2424  False  False  False    False     False  False          False
     2579  False  False  False    False     False  False          False
     2270  False  False  False    False     False  False          False
     1402  False  False  False    False     False  False          False
     652   False  False  False    False     False  False          False

[2768 rows x 7 columns]


<ipython-input-50-06fbda4bf1b6>:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  non_zero_non_one_counts = train_data.groupby('lang').apply(count_nonzero_nonone_rows)


In [51]:
# shuffle data
train_data=train_data.sample(frac=1)
# train_data=train_data.sample(100,random_state=42)


In [52]:
# train valid split
train,valid=train_test_split(train_data,test_size=0.2,random_state=42)

In [53]:
x_train, y_train = train['clean_message'].values.tolist(), train[label_columns].values.tolist()

x_valid, y_valid = valid['clean_message'].values.tolist(), valid[label_columns].values.tolist()

In [54]:
y_train_df = pd.DataFrame(y_train, columns=label_columns)

In [55]:
#update label column name

label_distribution = y_train_df.apply(pd.Series.value_counts).T.fillna(0).astype(int)

label_distribution.columns = ['count_0', 'count_1']
label_distribution['sum'] = label_distribution['count_0'] + label_distribution['count_1']
print(label_distribution)

          count_0  count_1   sum
anger        1939      275  2214
fear          909     1305  2214
joy          1685      529  2214
sadness      1504      710  2214
surprise     1545      669  2214


In [56]:
device="cuda" if torch.cuda.is_available() else 'cpu'

In [57]:
from sklearn.utils.class_weight import compute_class_weight

counts_0 = label_distribution['count_0'].to_numpy()
counts_1 = label_distribution['count_1'].to_numpy()

# Compute class-specific weights for each label (each class)
class_weights = []
for i in range(len(counts_0)):
    weight = compute_class_weight('balanced', classes=np.array([0, 1]), y=[0] * counts_0[i] + [1] * counts_1[i])
    class_weights.append(weight)

# Convert the list of weights to a tensor
class_weights_tensor = torch.tensor([w[1] for w in class_weights], dtype=torch.float,device=device)
class_weights_tensor

tensor([4.0255, 0.8483, 2.0926, 1.5592, 1.6547], device='cuda:0')

In [58]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, encodings, labels):

        self.encodings = encodings

        self.labels = labels



    def __getitem__(self, idx):

        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)

        return item


    def __len__(self):

        return len(self.labels)

In [59]:
!ls output/best-model-v1

ls: cannot access 'output/best-model-v1': No such file or directory


In [60]:
def build_dataset(x_train,y_train,x_valid,y_valid,MAX_LENGTH):

  train_encodings = tokenizer(x_train, truncation=True, padding="max_length", max_length=MAX_LENGTH)

  valid_encodings = tokenizer(x_valid, truncation=True, padding="max_length", max_length=MAX_LENGTH)

  train_dataset = Dataset(train_encodings, y_train)

  valid_dataset = Dataset(valid_encodings, y_valid)

  return train_dataset,valid_dataset



def train_arg():

    training_args = TrainingArguments(

    output_dir="./output/best-model-v1",

    report_to=None,
    dataloader_pin_memory=False,

    num_train_epochs=3,

    gradient_accumulation_steps=2,

    per_device_train_batch_size=8,

    per_device_eval_batch_size=16,

    learning_rate=2e-5,

    weight_decay=0.01,

    eval_strategy=IntervalStrategy.STEPS,

    save_total_limit=1,

    logging_dir="./logs",

    logging_strategy=IntervalStrategy.STEPS,

    logging_steps=250,

    eval_steps=500,

    load_best_model_at_end=True,

    lr_scheduler_type="cosine_with_restarts",

    warmup_steps=500,

    save_strategy="steps",

    save_steps=500,

  )

    return training_args

In [176]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(eval_pred):
    """
    Compute metrics for multi-label classification.

    Args:
        eval_pred: Tuple of (predictions, labels).
            predictions: logits or probabilities of shape (batch_size, num_labels)
            labels: ground truth of shape (batch_size, num_labels)

    Returns:
        metrics: Dictionary containing accuracy, precision, recall, and F1-score.
    """
    # Unpack predictions and labels
    logits, labels = eval_pred

    # Convert logits to binary predictions
    # If your model outputs probabilities (e.g., after sigmoid), use threshold=0.5
    probabilities = expit(logits)
    predictions = (probabilities > 0.5).astype(int)

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute precision, recall, f1-score
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average=None
    )

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [183]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from scipy.special import expit  # For sigmoid activation if needed

def evaluate_model(model, valid_dataset, compute_metrics, batch_size=32, device="cuda",average='None'):
    """
    Evaluate a multi-label classification model without using the Hugging Face Trainer.

    Args:
        model: The trained model (Hugging Face model).
        valid_dataset: The validation dataset (torch.utils.data.Dataset).
        compute_metrics: The function to compute evaluation metrics.
        batch_size: Batch size for evaluation.
        device: Device to perform evaluation (e.g., "cuda" or "cpu").

    Returns:
        metrics: Dictionary containing evaluation metrics.
    """
    # Set model to evaluation mode
    model.eval()
    model.to(device)

    # Create DataLoader for the validation dataset
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    all_predictions = []
    all_labels = []

    # Disable gradient computation for evaluation
    with torch.no_grad():
        for batch in tqdm(valid_loader, desc="Evaluating"):
            # Get inputs and labels
            inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
            labels = batch["labels"].to(device)

            # Forward pass: Get logits
            logits = model(**inputs)

            # Apply sigmoid activation to logits if necessary (for multi-label classification)
            probabilities = expit(logits.cpu().numpy())  # Convert logits to probabilities
            predictions = (probabilities > 0.5).astype(int)  # Convert probabilities to binary predictions

            # Collect predictions and labels
            all_predictions.append(predictions)
            all_labels.append(labels.cpu().numpy())
    print()
    # Concatenate all predictions and labels
    all_predictions = np.concatenate(all_predictions, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Unpack predictions and labels
    logits, labels = all_predictions,all_labels

    # Convert logits to binary predictions
    # If your model outputs probabilities (e.g., after sigmoid), use threshold=0.5
    probabilities = expit(logits)
    predictions = (probabilities > 0.5).astype(int)

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute precision, recall, f1-score
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average=average
    )
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    },all_predictions

In [184]:
def evaluate_func(trainer):
    metrics = evaluate_model(
    model=model,
    valid_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    batch_size=32,
    device=device
)
    accuracy=metrics["accuracy"]
    precision=metrics["precision"]
    recall=metrics["recall"]
    f1=metrics["f1"]

    return precision, recall, f1, accuracy

In [65]:
MAX_LENGTH = 256

metrics_df = pd.DataFrame(columns=["Models_name", "Precision", "Recall", "F1_score", "Accuracy"])

confusion_matrices = []

In [66]:
quantization_config = BitsAndBytesConfig(

    load_in_4bit=True,

    bnb_4bit_use_double_quant=True,

    bnb_4bit_quant_type="nf4",

    bnb_4bit_compute_dtype=torch.bfloat16,

)

In [67]:
models_list=models_list_3

models_name=models_name_3

x=models_list[0]

In [68]:
token=""

import huggingface_hub

if "gemma" in x or "Phi" in x:

  huggingface_hub.login(token=token)

In [69]:
USE_LORA=False

In [72]:
from transformers import AutoModel

In [73]:


torch.cuda.empty_cache()



print(x)
tokenizer = AutoTokenizer.from_pretrained(x)
embedding_model = AutoModel.from_pretrained(x, output_hidden_states=True)

FacebookAI/xlm-roberta-large


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [75]:
def find_target_modules(model):
    # Initialize a Set to Store Unique Layers
    unique_layers = set()

    # Iterate Over All Named Modules in the Model
    for name, module in model.named_modules():
        # Check if the Module Type Contains 'Linear4bit'
        if "Linear4bit" in str(type(module)):
            # Extract the Type of the Layer
            layer_type = name.split('.')[-1]

            # Add the Layer Type to the Set of Unique Layers
            unique_layers.add(layer_type)

    # Return the Set of Unique Layers Converted to a List
    return list(unique_layers)
find_target_modules(embedding_model)

[]

In [ ]:
# a=

In [76]:

# Define the MLP Classifier
class MLPClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()  # For multi-label classification

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return self.sigmoid(x)


In [99]:
# Define a custom loss function to handle class weights for multilabel classification
def custom_loss(output, target):
    # BCEWithLogitsLoss with pos_weight for each class
    loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=class_weights_tensor)
    return loss_fn(output, target)

In [100]:

# Combine XLM-RoBERTa with MLP
class XLMRobertaWithMLP(nn.Module):
    def __init__(self, embedding_model, mlp_model):
        super(XLMRobertaWithMLP, self).__init__()
        self.embedding_model = embedding_model
        for param in self.embedding_model.parameters():
            param.requires_grad = False
        self.mlp_model = mlp_model

    def forward(self, input_ids, attention_mask, labels=None):
        # Get embeddings from XLM-RoBERTa
        outputs = self.embedding_model(input_ids=input_ids, attention_mask=attention_mask)
        # Use the [CLS] token's hidden state
        cls_embedding = outputs.last_hidden_state[:, 0, :]
        # Pass embeddings to MLP
        logits=self.mlp_model(cls_embedding)
        if labels is not None:
            loss = custom_loss(logits, labels.float())
            return logits, loss
        return logits


In [101]:

# Define your MLP with appropriate dimensions
hidden_size = 1024  # For XLM-RoBERTa-large, hidden size = 1024
mlp_model = MLPClassifier(input_size=hidden_size, hidden_size=512, output_size=len(label_columns))

# Combine the models
model = XLMRobertaWithMLP(embedding_model=embedding_model, mlp_model=mlp_model)

In [102]:
embedding_model.eval() # embedding_model in training mode (dropout modules are activated)

if USE_LORA:

  # enable gradient check pointing

  embedding_model.gradient_checkpointing_enable()

  target_modules_dict={

      "microsoft/Phi-3.5-mini-instruct":'all-linear',
      "unsloth/Llama-3.2-1B-Instruct":["q_proj","k_proj","v_proj"],
      "unsloth/Qwen2.5-3B-Instruct-bnb-4bit":["q_proj","k_proj","v_proj"],
  }

  target_modules= target_modules_dict.get(x,["q_proj"])



  # enable quantized training

  embedding_model = prepare_model_for_kbit_training(embedding_model)

  config = LoraConfig(

    r=8,

    lora_alpha=32,

    # target_modules=["k_proj"],

    target_modules = target_modules,#TODOOOOOOOOOOOOO

    lora_dropout=0.1,

    bias="none",

    task_type="SEQ_CLS",

  )

  # LoRA trainable version of model

  embedding_model = get_peft_model(embedding_model, config)



  # trainable parameter count

  embedding_model.print_trainable_parameters()

device="cuda" if torch.cuda.is_available() else 'cpu'

In [103]:
if tokenizer.pad_token is None:
    print("adding tokenizer token")
    # tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    tokenizer.pad_token=tokenizer.eos_token

    embedding_model.config.pad_token_id = tokenizer.pad_token_id

    # model.resize_token_embeddings(len(tokenizer))

    if embedding_model.config.pad_token_id is None:
        print("setting model pad token")
        embedding_model.config.pad_token_id = tokenizer.pad_token_id

tokenizer.pad_token

'<pad>'

In [104]:
device

'cuda'

In [105]:
model = model.to(device)

for param in model.parameters():
    param.data = param.data.contiguous()


In [106]:
os.environ['WANDB_DISABLED']="true"
training_args = train_arg()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [165]:
train_dataset,valid_dataset=build_dataset(x_train,y_train,x_valid,y_valid,MAX_LENGTH)

In [111]:
# Define the Trainer with custom loss function
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False,num_items_in_batch=None):
        labels = inputs.get("labels")
        logits,loss = model(**inputs)
        loss = custom_loss(logits, labels)
        return (loss, logits) if return_outputs else loss

In [112]:
trainer = CustomTrainer(

    model=model,

    args=training_args,

    train_dataset=train_dataset,

    eval_dataset=valid_dataset,

    compute_metrics=compute_metrics,

    # data_collator=data_collator

)

In [113]:
checkpoints = sorted(glob.glob(os.path.join(trainer.args.output_dir, "checkpoint-*")), key=os.path.getmtime)



resume_checkpoint = True if checkpoints else False

embedding_model.config.use_cache = True
# embedding_model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

trainer.train(resume_from_checkpoint=resume_checkpoint)

# embedding_model.config.use_cache = True


Step,Training Loss,Validation Loss


TrainOutput(global_step=414, training_loss=1.789947509765625, metrics={'train_runtime': 322.3583, 'train_samples_per_second': 20.604, 'train_steps_per_second': 1.284, 'total_flos': 0.0, 'train_loss': 1.789947509765625, 'epoch': 2.9819494584837543})

In [160]:
trainer.compute_metrics=compute_metrics

In [ ]:
model(valid_dataset)

In [161]:
trainer.evaluate()

ValueError: Found input variables with inconsistent numbers of samples: [554, 519]

In [180]:
metrics,predicted_classes = evaluate_model(
  model=model,
  valid_dataset=valid_dataset,
  compute_metrics=compute_metrics,
  batch_size=32,
  device=device
)
accuracy=metrics["accuracy"]
precision=metrics["precision"]
recall=metrics["recall"]
f1=metrics["f1"]


new_data = pd.DataFrame({

  "Models_name": [x],

  "Precision": [precision],

  "Recall": [recall],

  "F1_score": [f1],

  "Accuracy": [accuracy]

})



metrics_df = pd.concat([metrics_df, new_data], ignore_index=True)



model_confusion_matrices = []








if isinstance(predicted_classes, tuple) :

  predicted_classes = predicted_classes[0]

predicted_classes = np.array(predicted_classes)

# print(f"Type after conversion: {type(predicted_classes)}")



if len(predicted_classes.shape) > 2:

  predicted_classes = predicted_classes.reshape(predicted_classes.shape[0], -1)

  # print(f"Shape after reshaping: {predicted_classes.shape}")









predicted_classes = (expit(predicted_classes) >= 0.5).astype(int)



y_valid_np = np.array(y_valid)



for j in range(len(label_columns)):

    print(f"Computing confusion matrix for label: {label_columns[j]}")

    cm = confusion_matrix(y_valid_np[:, j], predicted_classes[:, j])



    # cm = confusion_matrix(y_valid[:, j], predicted_classes[:, j])

    model_confusion_matrices.append(cm)

confusion_matrices.append(model_confusion_matrices)





print(f"{x} evaluates:")

for k, label in enumerate(label_columns):

  print(f"Class '{label}': Precision: {precision[k]:.4f}, Recall: {recall[k]:.4f}, F1: {f1[k]:.4f}")

print(f"Overall accuracy: {accuracy:.4f}")

print("-------------------------------------------------------------------------\n")

Evaluating: 100%|██████████| 18/18 [00:19<00:00,  1.10s/it]


Computing confusion matrix for label: anger
Computing confusion matrix for label: fear
Computing confusion matrix for label: joy
Computing confusion matrix for label: sadness
Computing confusion matrix for label: surprise
FacebookAI/xlm-roberta-large evaluates:
Class 'anger': Precision: 0.1047, Recall: 1.0000, F1: 0.1895
Class 'fear': Precision: 0.5523, Recall: 1.0000, F1: 0.7116
Class 'joy': Precision: 0.2617, Recall: 1.0000, F1: 0.4149
Class 'sadness': Precision: 0.3032, Recall: 1.0000, F1: 0.4654
Class 'surprise': Precision: 0.3069, Recall: 1.0000, F1: 0.4696
Overall accuracy: 0.0000
-------------------------------------------------------------------------



In [185]:
# Initialize a DataFrame to store the evaluation results for each language
lang_metrics_df = pd.DataFrame(columns=["Language", "Precision", "Recall", "F1_score", "Accuracy"])
# Temporarily disable metric computation
original_compute_metrics = trainer.compute_metrics
trainer.compute_metrics = None

# Iterate over each language
for lang in langs:
    # Filter the validation data for the current language
    lang_valid_data = valid[valid['lang'] == lang]
    lang_x_valid = lang_valid_data['clean_message'].values.tolist()
    lang_y_valid = lang_valid_data[label_columns].values.tolist()
    if(len(lang_valid_data)==0):
      continue
    # Tokenize the validation data
    lang_valid_encodings = tokenizer(lang_x_valid, truncation=True, padding="max_length", max_length=MAX_LENGTH)
    lang_valid_dataset = Dataset(lang_valid_encodings, lang_y_valid)

    metrics,_ = evaluate_model(
      model=model,
      valid_dataset=lang_valid_dataset,
      compute_metrics=compute_metrics,
      batch_size=32,
      device=device,
      average='weighted'
    )
    accuracy=metrics["accuracy"]
    precision=metrics["precision"]
    recall=metrics["recall"]
    f1=metrics["f1"]

    # Store the metrics in the DataFrame
    new_data = pd.DataFrame({
        "Language": [lang],
        "Precision": [precision],
        "Recall": [recall],
        "F1_score": [f1],
        "Accuracy": [accuracy]
    })

    lang_metrics_df = pd.concat([lang_metrics_df, new_data], ignore_index=True)

# Display the evaluation results for each language
print(lang_metrics_df)

# Restore compute_metrics function if needed
trainer.compute_metrics = original_compute_metrics

Evaluating: 100%|██████████| 18/18 [00:20<00:00,  1.12s/it]


  Language  Precision  Recall  F1_score  Accuracy
0      eng   0.373263     1.0  0.527658       0.0



<ipython-input-185-b06ec9ed045b>:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  lang_metrics_df = pd.concat([lang_metrics_df, new_data], ignore_index=True)


In [ ]:
a=

In [ ]:
import os
os.environ["HUGGING_FACE_HUB_TOKEN"] = token

In [ ]:
trainer.push_to_hub("roberta-sentiment")